This notebook allows you to run ACOLITE/RAdCor processing for Landsat OLI scenes on Google Colab. RAdCor is an atmospheric correction algorithm that can account for adjacency effects explicitly, by using realistic atmospheric point spread functions. RAdCor development was funded by the [BELSPO/STEREO programme](https://www.belspo.be/belspo/space/belspo_stereo_en.stm) under contract SR/00/406. For more information on RAdCor, check out the [website](https://odnature.naturalsciences.be/radcor/), [publication](https://doi.org/10.1364/AO.546766), and [ATBD](https://github.com/acolite/radcor_atbd).

To run the code in the notebook, click the execute button next to each cell to run that particular cell. Since cells use outputs from preceding cells, please run them in order.

The required dependencies are installed and the latest version of ACOLITE is cloned from [GitHub](https://github.com/acolite/acolite). The first run of a specific sensor will be slower as the sensor specific lookup tables (LUTs) are retrieved. Please note that all data (including LUTs) and installed software in the Colab instance are deleted after it expires. For more than occasional processing it is recommended to set up a local environment and install the software on your system.

In this example, Landsat scenes are retrieved from the USGS EarthExplorer and hence requires EarthExplorer credentials to be configured. If you have no such account, please register at [EarthExplorer](https://ers.cr.usgs.gov/register) and [generate an M2M Application Token](https://www.usgs.gov/media/files/m2m-application-token-documentation). Download speed of a scene to Google Drive will depend on the EarthExplorer connection, and typically takes a few minutes.

Optional ancillary data can be used if EarthData credentials are also set up. Register at the [EarthData Login](https://urs.earthdata.nasa.gov/) page and select OB.DAAC Data Access and LP DAAC Data Pool in your profile under Authorised Apps, Approve More Applications.

Author: Quinten Vanhellemont, RBINS, quinten.vanhellemont@naturalsciences.be
Created: 2025-07-01

In [ ]:
# general imports
import os, sys

The next cell installs ACOLITE dependencies using pip. The exclamation point before pip is a Jupyter notebook magic command that executes an operating system shell command directly from the notebook. We first try to import NetCDF, and assume if this fails that the dependencies have not been installed yet. This is a cell specific for colab, and would not typically be needed for local runs of ACOLITE.

In [ ]:
## try to import NetCDF4, under the assumption if this works the dependencies are installed
try:
  import netCDF4
except:
  !pip install numpy matplotlib scipy gdal pyproj scikit-image pyhdf pyresample netcdf4 h5py requests pygrib cartopy

The next cell imports the Python os package, and if needed clones the ACOLITE repository from GitHub. Again the "!" magic command is used to run in the command shell. This is a cell specific for colab, and would not typically be needed for local runs of ACOLITE.

In [ ]:
## check if we need to clone ACOLITE
if not os.path.exists('acolite'):
  !git clone --depth 1 https://github.com/acolite/acolite

The next cell adds the ACOLITE clone location to the system path, and imports the ACOLITE code.

In [ ]:
## add acolite to path and import
sys.path.append('acolite')
import acolite as ac

The next cell sets your EarthExplorer username (EARTHEXPLORER_u), password (EARTHEXPLORER_p) and token (EARTHEXPLORER_token) as environment variables, to be entered as strings.

In [ ]:
## EarthExplorer credentials are needed for retrieving data
## add EarthExplorer credentials to environment variables
os.environ['EARTHEXPLORER_u'] = ''
os.environ['EARTHEXPLORER_p'] = ''
os.environ['EARTHEXPLORER_token'] = ''

If you have an EarthData account, and want to use ancillary data you can input the EARTHDATA_u and EARTHDATA_p in a similar way. Don't forget to set  settings['ancillary_data'] = True in the settings dictionary later.

In [ ]:
## EarthData credentials are needed to retrieve ancillary data
## add EarthData credentials to environment variables
os.environ['EARTHDATA_u'] = ''
os.environ['EARTHDATA_p'] = ''

Now we will set up the location and date of interest. There are several ways to set this up in ACOLITE, but here we will use a longitude and latitude of the location. You can query a range of dates, but here we will use a single date.

Here we investigate "Round Lake", a small lake in Minnesota, about 50 km west of Duluth. The lake has a largest dimension of 2 km, and hence we expect adjacency effects, especially for turbid atmospheres. The lake is located in a vegetated area, and hence we expect largest adjacency effects in the NIR bands.

In [ ]:
## location of interest
region_name = 'RoundLake'
station_lon = -93.191
station_lat = 46.696
date = '2025-04-26'

The next cell queries EarthExplorer for Landsat-8/9 L1 OLI data for the given location and date.

In [ ]:
## use ACOLITE API to query EarthExplorer
entity_list, identifier_list, dataset_list, metadata_list = ac.api.earthexplorer.query(roi = 'POINT ({} {})'.format(station_lon, station_lat),
                                                                        level = 1, start_date = date,  end_date = date, attributes = True)
## print the results
for scene in identifier_list:
  print(scene)

 Since the lake is located in two WRS-2 tiles, to avoid downloading both, we choose one.

In [ ]:
## select one WRS-2 tile
tile = '027028'

## check which scenes to get
scene_idx = []
for i, scene in enumerate(identifier_list):
  if tile in scene:
    scene_idx.append(i)

## subset to selected tile
entity_list = [entity_list[i] for i in scene_idx]
identifier_list = [identifier_list[i] for i in scene_idx]
dataset_list = [dataset_list[i] for i in scene_idx]
metadata_list = [metadata_list[i] for i in scene_idx]

Now we will download the scene(s) from EarthExplorer to your Google Drive.

In [ ]:
## use ACOLITE API to download the scenes from EarthExplorer
local_scenes = ac.api.earthexplorer.download(entity_list, dataset_list, identifier_list, output = 'Input/')

To use ACOLITE from within Python, a settings dictionary is needed, with at minimum the inputfile and output path. We will also set a region of interest based on the longitude and latitude coordinates above, with an extent in kilometres. We select RAdCor processing, add a buffer that corresponds to the default RAdCor kernel radius, and crop to the centre area to remove this additional buffer. Feel free to experiment with enabling/disabling these options to see their impact.

Typically ACOLITE will generate a L1R NetCDF file, which is a direct conversion of the input data into a generic ACOLITE format, and a L2R file which contains the corrected data. In this example, the L1R NetCDF is deleted, and no L1R RGB files are output.

In [ ]:
# acolite settings
settings = {## basic input and output settings
            ## local_scenes list is used as returned from the API download function
            'inputfile': local_scenes,
            'output': 'Output/',

            ## set up region of interest centred on lon/lat, with box size in km
            'region_name': region_name,
            'station_lon': station_lon,
            'station_lat': station_lat,
            'station_box_size': 7,
            'station_box_units': 'km',

            ## disable ancillary data retrieval, set to True if EarthData credentials are set
            'ancillary_data': False,

            ## use RAdCor processing
            'atmospheric_correction_method': 'radcor',

            ## add a buffer and crop to centre to avoid masked data around the scene edges
            'limit_buffer': 5,
            'limit_buffer_units': 'km',
            'radcor_crop_centre': True,

            ## add the following to enable SWIR based glint correction
            #'dsf_residual_glint_correction': True,
            #'glint_mask_rhos_threshold': 0.10,

            ## delete L1R output NetCDF
            'l1r_delete_netcdf': True,
            ## don't output L1R RGBs
            'l1r_rgb_keys': [],}

Finally, ACOLITE processing can be started. Note that the first run for a given sensor will be slower as LUTs will need to be downloaded to your Google Drive. Output data will be stored in the Output folder, accessible from the sidebar to the left.

In [ ]:
## run acolite with settings dict
ret = ac.acolite.acolite_run(settings)

We can now extract a point of interest from the created L2R scene, and plot the results of RAdCor processing with and without taking the adjacency effects into account.

In [ ]:
## we can extract pixels for a box centred on the station
box_size = 11

## use the L2R files from each scene in the return dictionary
ext = {}
for n in ret:
  file = ret[n]['l2r'][0]
  print('Extracting data from {}'.format(file))
  ext[n] = ac.shared.nc_extract_point(file, station_lon, station_lat, box_size = box_size)

Now, let's plot the data. In the RAdCor processing, rhos represents the surface level reflectance under the heterogeneous surface assumption (i.e. with correction for adjacency effects), and rhosu is the reflectance under the homogeneous assumption (i.e. without correction for adjacency effects).

In [ ]:
## import the plotting function
import matplotlib.pyplot as plt

## make plot directory
if not os.path.exists('Plots'): os.makedirs('Plots')

## run through all extracted data
for n in ext:

  ## plot the mean reflectance in the first 5 bands
  plt.plot(ext[n]['rhos_wave'][0:5], [ext[n]['mean'][ds] for ds in ext[n]['rhos_datasets']][0:5],
          '.:', label = r'$\rho_{s}$')
  plt.plot(ext[n]['rhosu_wave'][0:5], [ext[n]['mean'][ds] for ds in ext[n]['rhosu_datasets']][0:5],
          '.:', label = r'$\rho_{su}$')

  ## plot y == 0 line
  xlim = plt.xlim()
  ylim = plt.ylim()
  plt.plot(xlim, [0,0], '--', color = 'Grey')
  plt.xlim(xlim)

  ## add legend and label axes and title
  plt.legend()
  plt.xlabel('Wavelength (nm)')
  plt.ylabel(r'$\rho_s$ (1)')
  plt.title('{} {}'.format(ext[n]['gatts']['sensor'].replace('_', '/'),
                          ext[n]['gatts']['isodate'][0:19]))

  ## save plot to PNG
  plt.savefig('Plots/{}_spectra.png'.format(ext[n]['gatts']['oname']),
              dpi = 300, facecolor = 'white', bbox_inches = 'tight')

  plt.show()

We can also plot the difference of reflectance in the NIR band not corrected/corrected for adjacency effects.

In [ ]:
## import numpy for math
import numpy as np

## wavelength of band to compare
wave = 865

## find the sensor specific band wavelength
widx = np.argsort(np.abs(np.asarray(ext[n]['rhos_wave'])-wave))[0]

## get rhos and rhosu datasets for this band
rhos_ds = ext[n]['rhos_datasets'][widx]
rhos_d = ac.shared.nc_data(file, rhos_ds)
rhosu_ds = ext[n]['rhosu_datasets'][widx]
rhosu_d = ac.shared.nc_data(file, rhosu_ds)

## compute difference of rhosu-rhos
ratio = rhosu_d-rhos_d

## plot image
plt.imshow(ratio, vmin = -0.015, vmax = 0.015, cmap = 'bwr')
plt.colorbar(label = 'difference {}-{} {} nm (1)'.format(r'$\rho_{su}$', r'$\rho_{s}$', wave, ))
plt.axis('off')
plt.title('{} {}'.format(ext[n]['gatts']['sensor'].replace('_', '/'),
                          ext[n]['gatts']['isodate'][0:19]))

## save plot to PNG
plt.savefig('Plots/{}_difference_{}.png'.format(ext[n]['gatts']['oname'], wave),
              dpi = 300, facecolor = 'white', bbox_inches = 'tight')

plt.show()